In [1]:
import numpy as np
from enum import Enum
from typing import List
import random
import math

# Things to do
- Initialize n agents, n/2 on each team (team A, B)
- start with 1 topic for opinions
- give uniform/normal dist of [-1, 0), (0, -1] for team A, B respectively
- set some tolerance threshold for if agents on opposing teams can work together
- make some payoff matrix with [0, 1] for probability of reproducing

In [2]:
class Team(Enum):
    A = 0
    B = 1

class Agent():
    def __init__(self, team, opinions):
        self.team: Team = team
        self.opinions: List = opinions
        self.probReproduce = 0
        self.childOpinions = []


In [3]:
TOLERANCE = 0.75
# [[(A, A), (A, B)]
#  [(B, A), (B, B)]]
UTILITY_MATRIX = [[1.0, 1.5],
                 [1.5, 1.0]]
def getUtility(agent1: Agent, agent2: Agent):
    return UTILITY_MATRIX[agent1.team.value][agent2.team.value]

In [4]:
numAgents = 100 # always even

def generateAgents(distribution: str) -> tuple[List[Agent], List[Agent]]:
    teamA = []
    teamB = []
    # [mean, std_dev, low bound, upper bound]
    teamADistribution = [-0.5, 0.25, -1, 0]
    teamBDistribution = [0.5, 0.25, 0, 1]

    if distribution == "normal":
        teamAOpinions = np.clip(np.random.normal(teamADistribution[0], teamADistribution[1], numAgents//2), teamADistribution[2], teamADistribution[3])
        teamBOpinions = np.clip(np.random.normal(teamBDistribution[0], teamBDistribution[1], numAgents//2), teamBDistribution[2], teamBDistribution[3])
    elif distribution == "uniform":
        teamAOpinions = np.clip(np.random.uniform(teamADistribution[2], teamADistribution[3], numAgents//2), teamADistribution[2], teamADistribution[3])
        teamBOpinions = np.clip(np.random.uniform(teamBDistribution[2], teamBDistribution[3], numAgents//2), teamBDistribution[2], teamBDistribution[3])

    print(teamAOpinions)
    print(teamBOpinions)
    for i in range(0, numAgents):
        if i < numAgents/2:
            # team A
            team = Team.A
            opinion = teamAOpinions[i]
            teamA.append(Agent(team, [opinion]))
        else:
            # team B
            team = Team.B
            opinion = teamBOpinions[i - numAgents//2]
            teamB.append(Agent(team, [opinion]))

    return teamA, teamB
        

In [5]:
teamA, teamB = generateAgents("normal")

[-0.00986015 -0.30088028 -0.64891264 -0.76365383 -0.82227026 -0.87851186
 -0.54365489 -0.21451644 -0.40336077 -0.54494815 -0.07941984 -0.43572726
 -0.45735104 -0.56855884 -0.89033998 -0.2129449  -0.55021892 -0.50994164
 -0.50758743 -0.51692672 -0.49797449 -0.71356842 -0.32510396 -0.53240757
 -0.57655618 -0.77593704 -0.53110813 -0.72510132  0.         -0.5363087
 -0.33350965 -0.43399737 -0.37337735 -0.8840108  -0.50240614 -0.75444957
 -0.04915235 -0.1821869  -0.07231024 -0.95991624 -0.46455996 -0.53890213
 -0.71615518 -0.04773003 -0.64526781 -1.         -0.73054444 -0.48115895
 -0.24184886 -0.54003185]
[4.99517359e-01 2.20407376e-01 5.53181466e-01 1.23850400e-01
 2.36460212e-01 3.92775494e-01 4.56308886e-01 7.78763182e-01
 4.35108691e-01 7.59027881e-01 5.80125401e-01 5.60517113e-01
 7.08525021e-01 3.44377242e-01 1.22288208e-01 9.31287391e-05
 5.43543795e-02 1.61133702e-01 5.19176762e-01 9.11127348e-01
 7.18399266e-01 4.19716451e-01 6.79928312e-01 4.74086252e-01
 4.19715186e-01 6.4216868

In [6]:
print(teamA[0].opinions[0])

-0.5735623537417095


In [7]:
# teamA, teamB = generateAgents("uniform")

In [8]:
def updateTeam(agents: List[Agent]):
    newTeam = []
    print(agents)
    team = agents[0].team
    for agent in agents:
        numChildren = int(agent.probReproduce)
        if random.random() < (agent.probReproduce - numChildren):
            numChildren += 1

        # print(numChildren)
        
        for i in range(0, numChildren):
            # make new agents
            newTeam.append(Agent(team, agent.childOpinions))

    return newTeam


In [9]:
FRACTION_MOVE = 0.25
def sim(teamA: List[Agent], teamB: List[Agent], numIterations: int):
    teamAUpdate = teamA
    teamBUpdate = teamB

    for i in range(0, numIterations):
        # combine both lists
        combinedAgents = teamAUpdate + teamBUpdate
        # shuffle the combined list
        random.shuffle(combinedAgents)
        # take pairs of agents from the combined list
        agentPairs = []
        while len(combinedAgents) >= 2: # if odd number the last agents just dies (maybe some other way to solve this)
            agentPairs.append((combinedAgents.pop(), combinedAgents.pop()))
        
        # check if agents can work together pair by pair - either same team auto yes, diff team check threshold, or both cases check threshold
        for pair in agentPairs:
            delta = abs(pair[0].opinions[0] - pair[1].opinions[0])
            print(f"same team: {pair[0].team == pair[1].team}, delta: {delta}")
            if delta <= TOLERANCE:
                # can work together
                utility = getUtility(pair[0], pair[1])
                # child opinions will move closer the midpoint between agents by the delta/4
                if pair[0].team != pair[1].team:
                    # diff teams, put a cap at 0
                    if pair[0].opinions[0] < pair[1].opinions[0]:
                        agent0ChildOpinions = min(0, pair[0].opinions[0] + delta*FRACTION_MOVE)
                        agent1ChildOpinions = max(0, pair[1].opinions[0] - delta*FRACTION_MOVE)
                    else:
                        agent0ChildOpinions = max(0, pair[0].opinions[0] - delta*FRACTION_MOVE)
                        agent1ChildOpinions = min(0, pair[1].opinions[0] + delta*FRACTION_MOVE)
                else:                
                    if pair[0].opinions[0] < pair[1].opinions[0]:
                        agent0ChildOpinions = pair[0].opinions[0] + delta*FRACTION_MOVE
                        agent1ChildOpinions = pair[0].opinions[0] - delta*FRACTION_MOVE
                    else:
                        agent0ChildOpinions = pair[0].opinions[0] - delta*FRACTION_MOVE
                        agent1ChildOpinions = pair[0].opinions[0] + delta*FRACTION_MOVE
            else:
                # can't work together
                utility = 0
                agent0ChildOpinions = []
                agent1ChildOpinions = []

            pair[0].probReproduce = utility
            pair[0].childOpinions = [agent0ChildOpinions]
            pair[1].probReproduce = utility
            pair[1].childOpinions = [agent1ChildOpinions]

            print(f"utility: {pair[0].probReproduce}")

        teamAUpdate = updateTeam(teamAUpdate)
        teamBUpdate = updateTeam(teamBUpdate)
                
        # if they can't then prob reproduce is 0
        # if they can then get prob reporduce from utility
        # calculate opinions of offspring
    return teamAUpdate, teamBUpdate

In [10]:
teamANew, teamBNew = sim(teamA, teamB, 10)

same team: True, delta: 0.15378070338941918
utility: 1.0
same team: False, delta: 1.2405994459387473
utility: 0
same team: False, delta: 1.6737218047163345
utility: 0
same team: True, delta: 0.28292583329131277
utility: 1.0
same team: False, delta: 1.4488760663876668
utility: 0
same team: True, delta: 0.3924327117366245
utility: 1.0
same team: False, delta: 0.8852648975381769
utility: 0
same team: False, delta: 1.1740966230746688
utility: 0
same team: True, delta: 0.0006467861111804085
utility: 1.0
same team: False, delta: 0.9501790757354617
utility: 0
same team: False, delta: 1.2023902149599377
utility: 0
same team: True, delta: 0.545760760073851
utility: 1.0
same team: False, delta: 1.1057827831210014
utility: 0
same team: False, delta: 1.1677635422528234
utility: 0
same team: False, delta: 0.6819798776941544
utility: 1.5
same team: True, delta: 0.3377528430121953
utility: 1.0
same team: False, delta: 1.3295319278686284
utility: 0
same team: False, delta: 1.0003491985546515
utility: 

In [11]:
print(f"teamASize: {len(teamANew)}, teamBSize: {len(teamBNew)}")

teamASize: 112, teamBSize: 111
